# 04장 연산증폭기 기본 회로

* 이름: 홍길동
* 학번: 2024123456
* 조: 1

## 공통 코드

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import io
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

from IPython.display import Image

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def tangent_line_end(f, x):
    h = 1e-4
    d = (f(x) - f(x-h)) / (h)
    return lambda t: d*t - d*x + f(x)

def tangent_line_and_gradient(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return (lambda t: d*t - d*x + f(x)), d

def tangent_line_and_gradient_end(f, x):
    h = 1e-4
    d = (f(x) - f(x-h)) / (h)
    return (lambda t: d*t - d*x + f(x)), d

def derivative(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return d

def get_simulation_result(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1

    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []

            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]

            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1

    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def print_value_to_string(label, value):
    output = io.StringIO()
    print('%s = ' % label, end='', file=output)
    print('{:11.3f}'.format(value), file=output)
    captured = output.getvalue()
    return captured

def draw_plot(xs, ys, label, style_idx, color_idx=-1, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if color_idx < 0:
        color_idx = style_idx

    if isinstance(xs, list):
        xs = np.array(xs)
    if isinstance(ys, list):
        ys = np.array(ys)

    if (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    elif not isinstance(xs, np.ndarray):
        selected_markevery = None
        selected_marker = markers[style_idx]
    elif (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[color_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[color_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 실험01 이득이-2인 반전증폭기

### 실험01-01 +15V와 -15V 만들기

#### 실험 방법

In [ ]:
display(Image("./images/ch04_ep01_01_circuit.png", width=300))

#### 실험 결과

##### 제작한 회로

In [ ]:
display(Image("./data/ch04_ep01_01_result_circuit.jpg", width=600))

In [4]:
# DMM으로 측정한 결과 기록

V_CH1_V = ?
V_CH2_V = ?

#### 결과 분석

In [ ]:
print_value("V_CH1_V", V_CH1_V)
print_value("V_CH2_V", V_CH2_V)

### 실험01-02~04 반전증폭기 만들기

#### 실험 방법

* 연산증폭기: 741 OP amp
* VCC: 15 V
* -VCC: -15 V
* Vi: 정현파, 주파수 1 kHz, 크기 1 Vpp

In [ ]:
display(Image("./images/ch04_ep01_02_circuit.png", width=400))

#### 실험 결과

##### 제작한 회로

In [ ]:
display(Image("./data/ch04_ep01_02_result_circuit.jpg", width=1000))

##### 오실로스코프 화면

In [ ]:
display(Image("./data/ch04_ep01_03_result.png", width=600))
display(Image("./data/ch04_ep01_04_result_xy.png", width=600))

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch04_ep01_02_result.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

# 입력 전압을 x축, 출력 전압을 y축으로 하는 그래프를 그린다.
plt.xlabel('V_IN_V')
plt.ylabel('V_OUT_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vins, vouts, "XY", 0)
plt.show()

V_IN_pp_V = vins.max() - vins.min()
print_value("V_IN_pp_V", V_IN_pp_V)
V_OUT_pp_V = vouts.max() - vouts.min()
print_value("V_OUT_pp_V", V_OUT_pp_V)

### 실험01-05 반전증폭기 출력 제한

#### 실험 방법

* Vi: 정현파, 주파수 1 kHz, 크기 20 Vpp

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display(Image("./data/ch04_ep01_05_result.png", width=600))
display(Image("./data/ch04_ep01_05_result_xy.png", width=600))

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch04_ep01_05_result.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

# 입력 전압을 x축, 출력 전압을 y축으로 하는 그래프를 그린다.
plt.xlabel('V_IN_V')
plt.ylabel('V_OUT_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vins, vouts, "XY", 0)
plt.show()


V_IN_pp_V = vins.max() - vins.min()
print_value("V_IN_pp_V", V_IN_pp_V)
V_OUT_pp_V = vouts.max() - vouts.min()
print_value("V_OUT_pp_V", V_OUT_pp_V)

### 실험01-06 반전증폭기 직류 증폭

#### 실험 방법

* Vi: 직류 1 V

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display(Image("./data/ch04_ep01_06_result.png", width=600))

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch04_ep01_06_result.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

V_IN_mean_V = vins.mean()
V_OUT_mean_V = vouts.mean()

print_value("V_IN_mean_V", V_IN_mean_V)
print_value("V_OUT_mean_V", V_OUT_mean_V)

*******************************************************************************
## 실험02 이득이 2인 비반전증폭기

### 실험02-01~02 비반전증폭기 만들기

#### 실험 방법

* 연산증폭기: 741 OP amp
* VCC: 15 V
* -VCC: -15 V
* Vi: 정현파, 주파수 1 kHz, 크기 1 Vpp

In [ ]:
display(Image("./images/ch04_ep02_01_circuit.png", width=400))

#### 실험 결과

##### 제작한 회로

In [ ]:
display(Image("./data/ch04_ep02_01_result_circuit.jpg", width=1000))

##### 오실로스코프 화면

In [ ]:
display(Image("./data/ch04_ep02_02_result.png", width=600))
display(Image("./data/ch04_ep02_02_result_xy.png", width=600))

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch04_ep02_02_result.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

# 입력 전압을 x축, 출력 전압을 y축으로 하는 그래프를 그린다.
plt.xlabel('V_IN_V')
plt.ylabel('V_OUT_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vins, vouts, "XY", 0)
plt.show()

V_IN_pp_V = vins.max() - vins.min()
print_value("V_IN_pp_V", V_IN_pp_V)
V_OUT_pp_V = vouts.max() - vouts.min()
print_value("V_OUT_pp_V", V_OUT_pp_V)

### 실험02-03 비반전증폭기 출력 제한

#### 실험 방법

* Vi: 정현파, 주파수 1 kHz, 크기 20 Vpp

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display(Image("./data/ch04_ep02_03_result.png", width=600))
display(Image("./data/ch04_ep02_03_result_xy.png", width=600))

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch04_ep02_03_result.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

# 입력 전압을 x축, 출력 전압을 y축으로 하는 그래프를 그린다.
plt.xlabel('V_IN_V')
plt.ylabel('V_OUT_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vins, vouts, "XY", 0)
plt.show()

V_IN_pp_V = vins.max() - vins.min()
print_value("V_IN_pp_V", V_IN_pp_V)
V_OUT_pp_V = vouts.max() - vouts.min()
print_value("V_OUT_pp_V", V_OUT_pp_V)

### 실험02-04 비반전증폭기 직류 증폭

#### 실험 방법

* Vi: 직류 1 V

#### 실험 결과

##### 오실로스코프 화면

In [ ]:

display(Image("./data/ch04_ep02_04_result.png", width=600))

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch04_ep02_04_result.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

V_IN_mean_V = vins.mean()
V_OUT_mean_V = vouts.mean()

print_value("V_IN_mean_V", V_IN_mean_V)
print_value("V_OUT_mean_V", V_OUT_mean_V)

*******************************************************************************
## 실험03 Voltage Follower

### 실험03-01 Voltage Follower 만들기

#### 실험 방법

* 연산증폭기: 741 OP amp
* VCC: 15 V
* -VCC: -15 V
* Vi: 정현파, 주파수 1 kHz, 크기 1 Vpp

In [ ]:
display(Image("./images/ch04_ep03_01_circuit.png", width=400))

#### 실험 결과

##### 제작한 회로

In [ ]:
display(Image("./data/ch04_ep03_01_result_circuit.jpg", width=1000))

##### 오실로스코프 화면

In [ ]:
display(Image("./data/ch04_ep03_01_result.png", width=600))
display(Image("./data/ch04_ep03_01_result_xy.png", width=600))

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch04_ep03_01_result.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

# 입력 전압을 x축, 출력 전압을 y축으로 하는 그래프를 그린다.
plt.xlabel('V_IN_V')
plt.ylabel('V_OUT_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vins, vouts, "XY", 0)
plt.show()

V_IN_pp_V = vins.max() - vins.min()
print_value("V_IN_pp_V", V_IN_pp_V)
V_OUT_pp_V = vouts.max() - vouts.min()
print_value("V_OUT_pp_V", V_OUT_pp_V)

*******************************************************************************
## 실험04 반전가산기

### 실험04-01~02 반전가산기 만들기

#### 실험 방법

* 연산증폭기: 741 OP amp
* VCC: 15 V
* -VCC: -15 V
* V1: 정현파, 주파수 1 kHz, 크기 2 Vpp
* V2: 구형파, 주파수 1 kHz, 크기 2 Vpp


In [ ]:
display(Image("./images/ch04_ep04_01_circuit.png", width=400))

#### 실험 결과

##### 제작한 회로

In [ ]:
display(Image("./data/ch04_ep04_01_result_circuit.jpg", width=1000))

##### 오실로스코프 화면

In [ ]:
display(Image("./data/ch04_ep04_02_result.png", width=600))

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch04_ep04_02_result.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

V_IN_pp_V = vins.max() - vins.min()
print_value("V_IN_pp_V", V_IN_pp_V)
V_OUT_pp_V = vouts.max() - vouts.min()
print_value("V_OUT_pp_V", V_OUT_pp_V)

*******************************************************************************
## 실험05 비반전가산기

### 실험05-01~02 비반전가산기 만들기

#### 실험 방법

* 연산증폭기: 741 OP amp
* VCC: 15 V
* -VCC: -15 V
* V1: 정현파, 주파수 1 kHz, 크기 2 Vpp
* V2: 구형파, 주파수 1 kHz, 크기 2 Vpp


In [ ]:
display(Image("./images/ch04_ep05_01_circuit.png", width=400))

#### 실험 결과

##### 제작한 회로

In [ ]:
display(Image("./data/ch04_ep05_01_result_circuit.jpg", width=1000))

##### 오실로스코프 화면

In [ ]:
display(Image("./data/ch04_ep05_02_result.png", width=600))

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch04_ep05_02_result.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()

V_IN_pp_V = vins.max() - vins.min()
print_value("V_IN_pp_V", V_IN_pp_V)
V_OUT_pp_V = vouts.max() - vouts.min()
print_value("V_OUT_pp_V", V_OUT_pp_V)

### 실험05-03 비반전가산기 실험2 

#### 실험 방법

* V1: 정현파, 주파수 1 kHz, 크기 3 Vpp
* V2: 구형파, 주파수 5 kHz, 크기 1 Vpp


#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display(Image("./data/ch04_ep05_03_result.png", width=600))

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch04_ep05_03_result.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
# vins = savgol_filter(vins, 100, 2)
# vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim((-0.0001, 0.0015))
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.show()